# MAKING PREDICTIONS HERE

In [1]:
### RUN THIS FIRST 
# Import dependencies
import numpy as np
import pandas as pd


# To create and fit model
from sklearn.model_selection import train_test_split
from sklearn import random_projection
from keras.models import Sequential, load_model
from keras.layers import Dense

# To transform data
from datetime import datetime as dt

# To save transformer
import pickle

# Opens model, transformer, and data point formatting
model_2 = load_model("RMA_Model.h5")
file = open('transformer.pickle', 'rb')
xform = pickle.load(file)
file.close()

Using TensorFlow backend.


In [2]:
### FUNCTION TO TRANSFORM INPUT AND PREDICT NEW POINTS
#Dependencies (Pandas, Keras.models (load_model), sklearn (random_projection))
def RMA_point(product_id, item_age):
    result = transform_point(empty_point(product_id, item_age))
    action = " "
    if result == 1:
        action = "Replace defective"
    else:
        action = "No replacement"
    return action, product_id, result

def transform_point(input_point):
    transformed_point = xform.transform(input_point)
    return(model_2.predict_classes(transformed_point)[0])

def empty_point(product_id, item_age):
    point = pd.read_csv('New_Point.csv')
    point = point.set_index('Unnamed: 0')
    point.loc['Lifespan(days)', "Encoding"] = item_age + 90
    point.loc[product_id, "Encoding"] = 1
    input_point = point['Encoding'].values.reshape(-1,2353)
    return input_point



In [9]:
### FUNCTION TO PREDICT MULTIPLE POINTS FROM CSV FORMAT
#Define function to count defectives in multiple point query
def RMA_multiple(csv_name):
    proxy_df = pd.read_csv(csv_name)
    defectives = []
    status = []
    proxy_list = []

    for row in proxy_df.index:
        result = RMA_point(proxy_df.iloc[row]["IB Material - Key"], proxy_df.iloc[row]['Lifespan(days)'])
        status.append(result[0])
        defectives.append(result[2])

    proxy_df['Status'] = status
    proxy_df['Defectives'] = defectives
    defective_count(proxy_df)
    return(proxy_df, "----", proxy_list)

def defective_count(proxy_df):

    for each in proxy_df.index:
        if proxy_df.iloc[each]['Status'] == 1:
            proxy_list.append([proxy_df.iloc[each]['IB Material - Key'], proxy_df.iloc[each]['Status']])


## Make Predictions HERE

In [ ]:
RMA_point()

In [12]:
RMA_multiple()

TypeError: RMA_multiple() missing 1 required positional argument: 'csv_name'

# EXAMPLES OF PREDICTIONS

## Single Point Examples

In [4]:
#Making random example single point predictions!!!!
RMA_point("XFP-10GE-SR", 100)

('No replacement', 'XFP-10GE-SR', 0)

In [5]:
#Making random example single point predictions!!!!
RMA_point("RE-400-768-R", 1235)

('Replace defective', 'RE-400-768-R', 1)

## Multiple Point Examples

In [8]:
#Making Multiple Point Predictions (4 data points) !!!!
RMA_multiple("proxy.csv")

(   Unnamed: 0 IB Material - Key  Lifespan(days)             Status  Defectives
 0           0       SRX110H2-VA           922.0  Replace defective           1
 1           1        EX3300-48P           138.0     No replacement           0
 2           2          SRX100H2           890.0     No replacement           0
 3           3  QFX5100-48S-3AFO           670.0     No replacement           0,
 '----',
 [])

# IMPORTANT LIMITATIONS

## input data must be be a certain age
## product models with low representation in dataset should be assessed with caution

### Note: 
Must consider way to use RMA_point() on full data_sets or CSV files
For loop with RMA_point()???
Data_set must have Product_ID and Item_Age
Item_age = Today() - Ship_Date